In [13]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats

In [14]:
fileName = '../../1000_population.tsv'


lines = []
categories = []


with open(fileName, 'r') as f:
    text = f.read()

lines = text.split('\n')
for l in lines:
    
    categories.append(l.split('\t')[-1])
allcategories=sorted(list(set(categories)))


In [15]:

def getAveragePer1000(filepath,n,filename):
    csv_df= pd.read_csv(filepath, low_memory = False)
    # csv_df = pd.DataFrame(csv_data)
    for col in allcategories:
        averages = [np.mean(csv_df.loc[i:i+n-1, col]) for i in range(0, csv_df.shape[0], n)]
        np.savetxt('../../similar_region_race_average100/'+col+"_"+filename+'.csv', averages, delimiter = ',')


In [16]:
def ifExistCSVFile_1000average_race(filename):
    filepath='../../similar_region_race_average100/'
    files=os.listdir(filepath)
    filenames=[]
    for col in allcategories:
        filename=col+"_"+filename+".csv"
        filenames.append(filename)
    for i in filenames:
        if i in files:
            return True
    return False

In [17]:

for curDir, dirs, files in os.walk(top="../../similar_region_race/"):
    for file in files:
        filename=file.split(".")[0]
        getAveragePer1000(filepath=curDir+file,n=1000,filename=file.split(".")[0])

In [18]:

arrforDistribution_different_race=dict()
for category in allcategories:
    arrforDistribution_different_race[category]=[]

for curDir, dirs, files in os.walk(top="../../similar_region_race_average100/"):
    for file in files:
        for category in allcategories:
            if file.startswith(category):
                csv_data=np.loadtxt(open(curDir+file,"rb"),delimiter=",",skiprows=0)
                arrforDistribution_different_race[category]+=csv_data.tolist()
                
        

In [19]:

mean_dict_different_race=dict()
std_dict_different_race=dict()
for category in allcategories:
    mean_dict_different_race[category]=0
    std_dict_different_race[category]=0

for category in allcategories:
    mean_dict_different_race[category]=np.mean(arrforDistribution_different_race[category])
    std_dict_different_race[category]=np.std(arrforDistribution_different_race[category])


In [20]:
mean_dict_different_race

{'ACB': -0.021129940274677197,
 'ASW': -0.020541364517036285,
 'BEB': -0.01690678177281044,
 'CDX': -0.01520099465128729,
 'CEU': -0.016533114461341817,
 'CHB': -0.015328450587149127,
 'CHS': -0.01553630769670323,
 'CLM': -0.017175762377800757,
 'ESN': -0.021289481956977087,
 'FIN': -0.016220012008386193,
 'GBR': -0.01616972796402513,
 'GIH': -0.016647828395962485,
 'GWD': -0.02127146261125899,
 'IBS': -0.016612028858784364,
 'ITU': -0.016483044568386403,
 'JPT': -0.015297644767223825,
 'KHV': -0.015510621661266998,
 'LWK': -0.020977118301868332,
 'MSL': -0.021221971390719675,
 'MXL': -0.016358508148574633,
 'PEL': -0.015192057641423692,
 'PJL': -0.016650973867877568,
 'PUR': -0.01772660210939756,
 'STU': -0.016488613724290634,
 'TSI': -0.016378152064038587,
 'YRI': -0.021395734138643974}

In [21]:
std_dict_different_race

{'ACB': 0.007866911384050993,
 'ASW': 0.007845222560823078,
 'BEB': 0.007884710711424594,
 'CDX': 0.007874313523803581,
 'CEU': 0.007927585629080875,
 'CHB': 0.007946756589325447,
 'CHS': 0.00794164324551959,
 'CLM': 0.007812970518334978,
 'ESN': 0.00785082325216948,
 'FIN': 0.007906254940044924,
 'GBR': 0.007932404777081037,
 'GIH': 0.007873289737229847,
 'GWD': 0.007853536883558044,
 'IBS': 0.007919548085211019,
 'ITU': 0.007768497466178242,
 'JPT': 0.00796914647357699,
 'KHV': 0.007990414654274063,
 'LWK': 0.007797858211516251,
 'MSL': 0.007864701129783158,
 'MXL': 0.007798067190318037,
 'PEL': 0.007722619015518075,
 'PJL': 0.007712973460516593,
 'PUR': 0.007785891217808583,
 'STU': 0.0077724707332740475,
 'TSI': 0.00791282517960449,
 'YRI': 0.007915268495722334}

In [22]:

def compareWith0_01Pvalue(mean_arr,std_arr,filename,filepath):

    arr=[]
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    for i in range(len(csv_matrix)):
        mid_res=stats.norm.cdf(csv_matrix[i], mean_arr, std_arr)

        if mid_res<0.05:
            arr.append(1)
        else:
            arr.append(0)
    np.savetxt("../../similar_region_race_comparewith_01/"+filename+".csv", arr, delimiter = ',')
    return filename

In [23]:
for curDir, dirs, files in os.walk(top="../../similar_region_race_average100/"):
    for file in files:   
        for category in allcategories:
            if file.startswith(category):
                filename=file[:file.index(".")]
                filepath=os.path.join(curDir,file)
                #print(filepath)
                compareWith0_01Pvalue(mean_arr=mean_dict_different_race[category],std_arr=std_dict_different_race[category],filename=filename,filepath=filepath)

In [24]:
def generatedfandcsv(columns,dic,filename):

    row_names=list(dic.keys())
    a=[]
    bs=[]
    #cs=[]
    for x in row_names:
        a.append(x)
        bs.append(dic[x])
        #cs.append(dic[x][1])
    dictforDF=dict()
    dictforDF[columns[0]]=a
    dictforDF[columns[1]]=bs
    #dictforDF[columns[2]]=cs

    df_index=pd.DataFrame(dictforDF)
    df_index.to_csv("../../similar_region_race_start_length/"+filename+"_start_lenth.csv")
    return df_index


In [25]:
def checkContinuous1(arr):
    res=dict()
    i=0
    while i<len(arr)-1:
        if arr[i]==1:
            count=1
            for j in range(i+1,len(arr),1):
                if arr[j]==1:
                    count+=1                    
                else:                   
                    break
            res[i]=count
            i+=count
        else:
            i+=1
            continue
    return res
            


In [26]:
def getstartAndLength(filepath,filename):
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    res=checkContinuous1(csv_matrix)
    df=generatedfandcsv(columns=["start","length"],dic=res,filename=filename)
    return df

In [27]:
for curDir, dirs, files in os.walk(top="../../similar_region_race_comparewith_01/"):
    for file in files:

        filename=file[:file.index(".")]
        filepath=os.path.join(curDir,file)
        getstartAndLength(filepath=filepath,filename=filename)

In [28]:

def countNumofLength(dic,filepath):
    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr=np.array(csv_df['length'])
    #print(dic)
    for i in range(len(arr)):
        #print("before iteration",dic)
        if arr[i] in dic:
            #print("in iteration not",dic)
            dic[arr[i]]+=1
        else:
            dic[arr[i]]=1
            #print("in iteration",dic)
    #print(dic)
    dic = {k: v for k, v in sorted(dic.items())}

    return dic

In [29]:

race_dic_except_mhc=dict()
for category in allcategories:
    race_dic_except_mhc[category]=dict()

for curDir, dirs, files in os.walk(top="../../similar_region_race_start_length/"):
    for category in allcategories:
        mydic=dict()
        for file in files:        
            if file.startswith(category):
                filename=file[:file.index(".")]
                if "mhc" in filename:
                    continue          
                else:                  
                    filepath=os.path.join(curDir,file)
                    mydic=countNumofLength(mydic,filepath=filepath)
        race_dic_except_mhc[category]=mydic

In [30]:
race_dic_except_mhc

{'ACB': {1: 2157, 2: 429, 3: 119, 4: 35, 5: 4, 6: 6, 7: 1},
 'ASW': {1: 2187, 2: 441, 3: 105, 4: 23, 5: 7, 6: 4, 7: 2},
 'BEB': {1: 2134, 2: 446, 3: 103, 4: 29, 5: 7, 6: 5, 7: 3, 8: 1},
 'CDX': {1: 2128, 2: 482, 3: 107, 4: 26, 5: 15, 6: 3, 7: 2, 8: 1},
 'CEU': {1: 2135, 2: 434, 3: 98, 4: 34, 5: 5, 6: 4, 7: 2, 17: 1},
 'CHB': {1: 2120, 2: 459, 3: 115, 4: 27, 5: 9, 6: 7, 7: 2},
 'CHS': {1: 2129, 2: 451, 3: 103, 4: 37, 5: 12, 6: 3, 8: 1},
 'CLM': {1: 2119, 2: 416, 3: 102, 4: 28, 5: 11, 6: 4, 7: 3, 8: 1, 10: 1},
 'ESN': {1: 2197, 2: 447, 3: 121, 4: 24, 5: 4, 6: 6, 7: 3, 8: 1},
 'FIN': {1: 2180, 2: 447, 3: 123, 4: 20, 5: 12, 6: 3},
 'GBR': {1: 2145, 2: 422, 3: 102, 4: 22, 5: 6, 6: 6, 7: 2, 8: 1, 9: 1, 17: 1},
 'GIH': {1: 2180, 2: 433, 3: 114, 4: 32, 5: 8, 6: 5, 9: 1},
 'GWD': {1: 2156, 2: 449, 3: 117, 4: 27, 5: 8, 6: 4, 7: 1},
 'IBS': {1: 2092, 2: 442, 3: 90, 4: 39, 5: 8, 6: 3, 7: 2, 8: 1, 17: 1},
 'ITU': {1: 2101, 2: 476, 3: 89, 4: 36, 5: 8, 6: 3, 7: 1, 8: 1},
 'JPT': {1: 2127, 2: 453, 3: 

In [31]:
mhc_final_dict=dict()
for curDir, dirs, files in os.walk(top="../../similar_region_race_start_length/"):
    for category in allcategories:
        mhcdic=dict()
        for file in files:   
            if file.startswith(category):
                filename=file[:file.index(".")]
                if "mhc" in filename:
                    filepath=os.path.join(curDir,file)
                    mhcdic=countNumofLength(mhcdic,filepath=filepath)
        mhc_final_dict[category]=mhcdic
                  

In [32]:
mhc_final_dict

{'ACB': {1: 3, 3: 1, 5: 1, 15: 1, 21: 1, 28: 1},
 'ASW': {1: 3, 3: 1, 5: 1, 8: 1, 9: 1, 19: 1, 28: 1},
 'BEB': {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 16: 1, 21: 1, 28: 1},
 'CDX': {1: 5, 3: 1, 4: 2, 5: 1, 21: 1, 29: 1},
 'CEU': {2: 2, 3: 3, 5: 1, 7: 1, 21: 1, 28: 1},
 'CHB': {1: 1, 2: 1, 3: 1, 5: 2, 6: 1, 9: 1, 21: 1, 28: 1},
 'CHS': {1: 2, 2: 1, 3: 1, 4: 3, 5: 1, 21: 1, 23: 1},
 'CLM': {1: 2, 2: 2, 3: 1, 5: 2, 8: 1, 21: 1, 28: 1},
 'ESN': {1: 2, 3: 1, 4: 1, 5: 1, 18: 1, 21: 1, 23: 1},
 'FIN': {1: 2, 2: 1, 3: 3, 4: 1, 5: 1, 21: 1, 28: 1},
 'GBR': {1: 2, 2: 1, 3: 5, 5: 1, 21: 1, 28: 1},
 'GIH': {1: 3, 2: 1, 3: 1, 4: 1, 5: 1, 8: 1, 21: 1, 28: 1},
 'GWD': {1: 3, 3: 1, 5: 1, 18: 1, 19: 1, 28: 1},
 'IBS': {1: 1, 2: 2, 3: 2, 5: 1, 8: 1, 21: 1, 28: 1},
 'ITU': {2: 3, 3: 1, 5: 2, 8: 1, 21: 1, 29: 1},
 'JPT': {1: 2, 4: 1, 5: 1, 8: 1, 9: 1, 21: 1, 28: 1},
 'KHV': {1: 2, 3: 1, 4: 1, 5: 1, 18: 1, 21: 1, 24: 1},
 'LWK': {1: 3, 2: 1, 3: 1, 5: 1, 15: 1, 19: 1, 29: 1},
 'MSL': {1: 3, 3: 1, 5: 1, 15: 1, 21: 1,

In [33]:
arr1=[]
for _,v in race_dic_except_mhc.items():
    for k2,arr in v.items():
        for i in range(arr):
            arr1.append(k2)
for _,v in mhc_final_dict.items():
    for k2,arr in v.items():
        for i in range(arr):
            arr1.append(k2)

In [34]:
mean_1=np.mean(arr1)
dev1=np.std(arr1)
print("mean",mean_1)
print("dev",dev1)
mid_res=stats.norm.ppf(0.9999999999,mean_1,dev1)
mid_res

mean 1.3221731596892292
dev 0.9523612770304682


7.380467893027601

In [35]:
def WithThreshold(threshold,dic):#dic={asian:{1:2,2:3,3:1},europe:{1:2,2:3,3:1}}
    larger_than_thres=0
    resultdic=dict()
    for category in allcategories:
        for i in dic[category].keys():
            if i>=threshold:
                larger_than_thres+=dic[category][i]
            resultdic[category]=larger_than_thres
    return resultdic


In [36]:

def getAllPositions(filepath):
    pos=[]
    #vcf_in=vcf(filepath)
    vcf_file = pysam.VariantFile(filepath)
    for record in vcf_file:

        pos.append(record.pos)
    pos=sorted(list(set(pos)))
    return pos

In [37]:
def findPosition(filepath,start,length):
    arr=[]
    allpos=getAllPositions(filepath=filepath)
    arr.append(allpos[start*1000])
    if (start+length)*1000-1<len(allpos):
        end=allpos[(start+length)*1000-1]
    else:
        end=allpos[-1]
    arr.append(end)
    return arr



In [38]:
filepath="../../similar_region_race_start_length/ACB_split_chr1_xaa_start_lenth.csv"
filename_components=(filepath.split("/")[-1]).split("_start_lenth.csv")[0].split("_")
filename_components

['ACB', 'split', 'chr1', 'xaa']

In [39]:

def GetPosition(filepath,threshold,res):

    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]

    filename_components=(filepath.split("/")[-1]).split("_start_lenth.csv")[0].split("_")
    #print(filename_components)
    findfoldername="split_"+filename_components[2]
    findfoldername_filename=filename_components[3]
    filepath="/data2/wangxuedong/mhc_test_data/"+findfoldername+"/"+findfoldername_filename+".vcf.gz"
    print(filepath)
    for k,v in mydic.items():
        if v>=threshold:
            print("filepath",filepath)
            midarr.append(findPosition(filepath=filepath,start=k,length=v))
            
        else:
            continue
    # category=filepath.split("/")[2].split("_")
    if midarr!=[]:
        res[findfoldername+"/"+findfoldername_filename]=midarr
    return res

In [40]:

def GetPositionformhc(filepath,threshold,res):

    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]
        
    filepath="/data2/wangxuedong/mhc_test_data/mhcdataset/1kgp.29720000-33130000.vcf"

    for k,v in mydic.items():
        if v>=threshold:
            print("filepath",filepath)
            midarr.append(findPosition(filepath=filepath,start=k,length=v))
            
        else:
            continue
    # category=filepath.split("/")[2].split("_")
    if midarr!=[]:
        res["mhc"]=midarr
    return res

In [41]:
all_race_positons=dict()
for category in allcategories:
    all_race_positons[category]=dict()

for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/similar_region_race_start_length/"):        
    for category in allcategories:
        res=dict()
        for file in files:
            filename=file[:file.index(".")]
            if "mhc" in filename:
                continue
            else:
                filerace=filename.split("_")[0]
                if filerace==category:                   
                    filepath=os.path.join(curDir,file)
                    res=GetPosition(filepath=filepath,threshold=8,res=res)
                    all_race_positons[category]=res


/data2/wangxuedong/mhc_test_data/split_chr1/xag.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xaj.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xah.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xak.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xaf.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xac.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xar.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xam.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xal.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xab.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xaa.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xad.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xap.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xan.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xai.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xao.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xae.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xaq.vcf.gz
/data2/wan

[E::idx_find_and_load] Could not retrieve index file for '/data2/wangxuedong/mhc_test_data/split_chr3/xav.vcf.gz'


/data2/wangxuedong/mhc_test_data/split_chr3/xaq.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xas.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xau.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xan.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xap.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xao.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xak.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xal.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xaj.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xae.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xaf.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xag.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xab.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr22/xag.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xac.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xai.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xag.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr1/xaj.vcf.gz
/data2/wa

[E::idx_find_and_load] Could not retrieve index file for '/data2/wangxuedong/mhc_test_data/split_chr13/xai.vcf.gz'


/data2/wangxuedong/mhc_test_data/split_chr13/xak.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xaj.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xav.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xam.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xal.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xac.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xap.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xae.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xaf.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xad.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xar.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xag.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xaq.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xao.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xas.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xan.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xat.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xaa

...

/data2/wangxuedong/mhc_test_data/split_chr13/xak.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xaj.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xav.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xam.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xal.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xac.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xap.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xae.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xaf.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xad.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xar.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xag.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xaq.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xao.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xas.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xan.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr13/xat.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr14/xaa

[E::idx_find_and_load] Could not retrieve index file for '/data2/wangxuedong/mhc_test_data/split_chr3/xav.vcf.gz'


/data2/wangxuedong/mhc_test_data/split_chr3/xax.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xbc.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xbb.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xbr.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xar.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xaw.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xat.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xaq.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xas.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xam.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xan.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xap.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr22/xai.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xah.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xak.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xal.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xai.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr3/xae.vcf.gz
/data2/wa

[E::idx_find_and_load] Could not retrieve index file for '/data2/wangxuedong/mhc_test_data/split_chr17/xaj.vcf.gz'


/data2/wangxuedong/mhc_test_data/split_chr16/xar.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr16/xaq.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xab.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xac.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xam.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xah.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xag.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xak.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xai.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xad.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xaf.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xar.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xae.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xan.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xal.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xaq.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xao.vcf.gz
/data2/wangxuedong/mhc_test_data/split_chr17/xap

In [42]:
with open('/data2/wangxuedong/mhc_test_data/similarity_region_pythonfiles/Firstpart/regions/similar_region_race_final.csv', 'w') as f:
    [f.write('{0},{1}\n'.format(key, value)) for key, value in all_race_positons.items()]


In [43]:
all_race_positons

{'ACB': {},
 'ASW': {},
 'BEB': {'split_chr3/xav': [[98046342, 98273693]]},
 'CDX': {'split_chr13/xai': [[55102733, 55443491]]},
 'CEU': {'split_chr17/xaj': [[45670177, 46418024]]},
 'CHB': {},
 'CHS': {'split_chr3/xav': [[98046342, 98273693]]},
 'CLM': {'split_chr17/xaj': [[45836662, 46224960]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'ESN': {'split_chr5/xai': [[34264948, 34562837]]},
 'FIN': {},
 'GBR': {'split_chr1/xbt': [[226978066, 227352188]],
  'split_chr17/xaj': [[45670177, 46418024]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'GIH': {'split_chr3/xav': [[98046342, 98276757]]},
 'GWD': {},
 'IBS': {'split_chr17/xaj': [[45670177, 46418024]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'ITU': {'split_chr17/xak': [[52575391, 52889013]]},
 'JPT': {},
 'KHV': {},
 'LWK': {},
 'MSL': {'split_chr5/xai': [[34264948, 34562837]]},
 'MXL': {'split_chr1/xbt': [[227016487, 227352188]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'PEL': {'split_chr11/xaf': [[23168664, 23564494]]

In [44]:
mhc_race_positons=dict()
for category in allcategories:
    mhc_race_positons[category]=dict()

for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/similar_region_race_start_length/"):        
    for category in allcategories:
        res=dict()
        for file in files:
            filename=file[:file.index(".")]
            if "mhc" in filename:
                filerace=filename.split("_")[0]
                if filerace==category:                   
                    filepath=os.path.join(curDir,file)
                    print(filepath)
                    res=GetPositionformhc(filepath=filepath,threshold=8,res=res)
                    mhc_race_positons[category]=res
                else:
                    continue
            else:
                continue


/data2/wangxuedong/mhc_test_data/similar_region_race_start_length/ACB_mhc_start_lenth.csv
filepath /data2/wangxuedong/mhc_test_data/mhcdataset/1kgp.29720000-33130000.vcf
filepath /data2/wangxuedong/mhc_test_data/mhcdataset/1kgp.29720000-33130000.vcf
filepath /data2/wangxuedong/mhc_test_data/mhcdataset/1kgp.29720000-33130000.vcf
/data2/wangxuedong/mhc_test_data/similar_region_race_start_length/ASW_mhc_start_lenth.csv
filepath /data2/wangxuedong/mhc_test_data/mhcdataset/1kgp.29720000-33130000.vcf
filepath /data2/wangxuedong/mhc_test_data/mhcdataset/1kgp.29720000-33130000.vcf
filepath /data2/wangxuedong/mhc_test_data/mhcdataset/1kgp.29720000-33130000.vcf
filepath /data2/wangxuedong/mhc_test_data/mhcdataset/1kgp.29720000-33130000.vcf
/data2/wangxuedong/mhc_test_data/similar_region_race_start_length/BEB_mhc_start_lenth.csv
filepath /data2/wangxuedong/mhc_test_data/mhcdataset/1kgp.29720000-33130000.vcf
filepath /data2/wangxuedong/mhc_test_data/mhcdataset/1kgp.29720000-33130000.vcf
filepath /

In [45]:
mhc_race_positons

{'ACB': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'ASW': {'mhc': [[29720403, 29913914],
   [29939668, 30120966],
   [31052133, 31528792],
   [32212726, 32882258]]},
 'BEB': {'mhc': [[29720403, 30048796],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'CDX': {'mhc': [[30994370, 31528792], [32212726, 32923168]]},
 'CEU': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'CHB': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'CHS': {'mhc': [[30994370, 31528792], [32397207, 32882258]]},
 'CLM': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'ESN': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32397207, 32882258]]},
 'FIN': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'GBR': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'GIH': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'GWD': {'mhc': [[29720